In [1]:
import postgresql.connection as connection
import postgresql.connect_info as connect_info
import psycopg

In [2]:
def exec_sql(sql, data_tuple=None):
    conn_info = connect_info.get_conninfo()
    with psycopg.connect(conn_info) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, data_tuple)
            try:
                result = cur.fetchall()
            except:
                result = None
    return result

# Fist Lmabda output list[tuple(title, content, uid)]

In [3]:
def get_title_content_uid_tuple_a_date(from_date_include, to_date_exclude):
# date like '2023-01-01'
    sql = f"""
        SELECT time, title, content, uid FROM news_data_select
        WHERE '{from_date_include}' <= time AND time < '{to_date_exclude}';
        """

    result = exec_sql(sql)

    return result

In [4]:
all_news = get_title_content_uid_tuple_a_date('2023-01-01', '2023-01-02')
all_news[0]

(datetime.datetime(2023, 1, 1, 8, 44),
 '黃偉哲元旦宣布生育、青年首購屋與創業補助加碼',
 ['〔記者王涵平／台南報導〕台南市長黃偉哲今宣布社福利多，生育獎勵金、青年首購屋與創業補助加碼，希望減輕市民負擔，市府團隊在新的一年也能加強對民眾的服務。',
  '黃偉哲今天參加在南瀛綠都心舉辦的升旗典禮與健走活動，宣布生育獎勵金在第一胎跟第二胎提高到2萬元，此外，對於年輕人首購屋的利息補貼增加，青年創業貸款的額度增加，前兩年的利息全免。',
  '黃偉哲表示，台南防疫持續維持六都確診人數、確診率、重症人數、重症率最低的城市，這些年也看到台灣面對疫情所展現的韌性，在邁入疫後新生活之際，要如何維持社會活力以及整個經濟動能，一直是我們中心思考的議題。',
  '他說，針對超徵稅賦，中央會有多少撥到地方還不知道，但南市優先宣布幾項社福措施，主要希望市民的負擔能夠減輕，也期勉市府團隊更加努力對於市民的需要加強協助服務。'],
 UUID('bc9b32ea-896f-11ed-aa4c-65ecba92a6aa'))

# second lambda save vector

In [5]:
def create_validate_content(content):
    import re
    result_content = []
    for paragraph in content:
        paragraph = re.sub(r'〔.*?〕', '', paragraph)
        paragraph = paragraph.replace('\n', '').replace(' ', '')

        if len(paragraph) > 0:
            result_content.append(paragraph)
    
    return result_content

def create_sbert_input(title, content):
    sbert_input = title + ''.join(content[:2])
    sbert_input.replace('\n', ' ')
    return sbert_input

from sentence_transformers import SentenceTransformer


def create_sbert_embedding(sbert_input):
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    emb = model.encode(sbert_input, convert_to_tensor=True)
    return emb

In [6]:
import pickle

def news_uid_sbert_embedding_to_db(time, news_uid, sbert_embedding):

    sql = """
        CREATE TABLE if NOT EXISTS news_sbert_embedding (
            time timestamp,
            news_uid uuid,
            sbert_embedding bytea,
            primary key (news_uid)
        );
        """
    
    exec_sql(sql)

    tensor_bytes = pickle.dumps(sbert_embedding)

    sql = f"""
        INSERT INTO news_sbert_embedding (time, news_uid, sbert_embedding) VALUES (%s, %s, %s);
        """
    
    exec_sql(sql, (time, news_uid, tensor_bytes))

In [7]:
def one_news_to_vector(time, title, content, uid):
    content = create_validate_content(content)
    sbert_input = create_sbert_input(title, content)
    sbert_embedding = create_sbert_embedding(sbert_input)
    news_uid_sbert_embedding_to_db(time, uid, sbert_embedding)
    print(f"{title} done!")

In [9]:
import tqdm.notebook as tqdm

for news in tqdm.tqdm(all_news):
    time, title, content, uid = news
    one_news_to_vector(time, title, content, uid)

  0%|          | 0/217 [00:00<?, ?it/s]

黃偉哲元旦宣布生育、青年首購屋與創業補助加碼 done!
國民黨元旦升旗 馬英九「噹」朱立倫議長選舉要檢討 done!
跨年到升旗間隔太近 陳光復：明年元旦升旗延後不強迫參加 done!
新竹縣2023元旦升旗 楊文科接下反生命園區自救會陳情書 done!
郭台銘傳「同舟」回國民黨 朱立倫：大門永遠敞開 done!
六大核心戰略產業階段檢討 蔡總統：近期召開諮詢會議 done!
女性是否服兵役 黃偉哲盼多徵詢各方意見  done!
國家太空中心升格行政法人   英文名增加「Taiwan」提升國際識別度 done!
民進黨九合一敗選檢討 蔡英文：黨務、政務都跟我有關 done!
新北元旦升旗典禮 侯友宜：成為和平安定、繁榮永續的國家 done!
上任第一個元旦升旗 張善政宣布明年起13個行政區比照辦理 done!
張善政：明年桃園13區公所都辦升旗典禮 done!
元旦談話揭4項新任務 蔡總統：團結一心讓國家平安幸福 done!
拍馬屁？桃市升旗典禮取「善好」 張善政：大家都喜歡 done!
柯文哲談敬老金「天下沒白吃的午餐」 蔣萬安：依規定辦理 done!
傳國民黨三月啟動總統初選 侯友宜：扮演好自己的角色 done!
彰化元旦升旗不一樣！ 世界最快超輕型飛機「鯊魚競賽機」亮相 done!
抗中保台變「和平保台」？ 柯文哲酸：民進黨沒有中心思想 done!
1800億稅收擬全民共享  侯友宜：振興庶民經濟 done!
感念亡故榮民保家衛國 馮世寬赴岡山榮家榮靈祠祭祀 done!
王鴻薇批綠營操作「抗中保農」 吳怡農：堅持反共保台 done!
倒數黃金週 吳怡農鬆口：1/4與賴清德合體 done!
陳其邁叫錯議長！ 康裕成問「怎麼修理？」 網友：罰唱紅豆 done!
黑統爭議？ 葉林傳、季麟連缺席元旦升旗 吳怡農：不必批評對手 done!
遭點名想選2024黨主席也擋不住 侯友宜：每個人都有本分 done!
獨家》共機去年進入我空域271天 廣播驅離增4成 done!
1800億與民共享 王鴻薇：就是我提的還稅於民 done!
議員質疑「新北幣」成效不彰 市府：試辦階段配合活動推廣 done!
中共軍機續擾台  15架次越台海中線、進西南空域 done!
農業勞動力變老又缺工 監院籲正視移工政策等問題 done!
蔡英文不發現金籲民眾理解 管碧玲：人民感受失落多於體諒

In [14]:
def get_200_opinion_extract_news():
    sql = f"""
        SELECT DISTINCT ON (n.uid) n.time, n.title, n.content, n.uid
        FROM news_data_select AS n
        INNER JOIN opinion_extract_200_news AS o
        ON n.uid=o.news_uid;
        """

    result = exec_sql(sql)
    return result

In [15]:
all_opinion_news = get_200_opinion_extract_news()
all_opinion_news[0]

(datetime.datetime(2023, 1, 12, 22, 38),
 '師生開戰！余正煌律師團：陳明通以私害公有失人師風範',
 ['〔記者張文川／台北報導〕調查局調查官余正煌自訴控告前新竹市長林智堅涉違反著作權法，昨天（11日）開庭後餘波蕩漾，論文指導教授陳明通今天發聲明力挺林智堅，余正煌今晚也由律師團發出聲明反擊指出，陳明通身為國安局長卻以私害公，有違國安局長重責，也有失人師風範，務請三思。',
  '余正煌的律師聲明指出，余正煌之碩士論文均為自行撰寫完成，故不存在任何部分為抄襲林智堅所稱「公版」之情形。',
  '其次，林智堅所主張之2016年2月1日所撰寫之文字，均為余正煌所撰寫後交給陳明通，然為避免被他人抄襲而將以APA註釋部分移除，故林智堅105年2月1日所稱之初稿，均無任何APA註釋。',
  '聲明中說，陳明通今年1月12日所發聲明稿，稱「只是余正煌增加一點文獻檢討，並引註」云云，純屬無稽。蓋凡寫過文章及論文者均知，一篇沒有任何註釋的文章，是不可能知悉該段文字之出處，更不可能大海撈針地把引用文章找出來，此乃常識問題，不懂身為一個教授、且貴為國安局長之人，竟然提出此種看法。',
  '律師團指出，陳明通為本案利害關係人，更事涉自身退休前擔任台大教職之正當性，本應尊重台大內部調查，並靜候法院審理釐清，自不應為左袒林智堅，一再書立聲明稿、透過媒體放話，而有失身為人師應有之風範格調。',
  '退萬步而言，就算陳明通主張2016年2月1日來源為陳明通撰寫之「公版」，而此「公版」整篇欠缺註釋下，身為臺灣大學資深教授，豈非自認大量抄襲其他學者之著作而未有引註，如此說法顯然荒謬至極。',
  '聲明最後強調，陳明通身為國安局長，在國際情勢動盪、兩岸情勢不穩，卻以私害公為了林智堅的論文，傾全力對付余正煌，恐與身兼國安局長重責大任有違，務請三思。'],
 UUID('00f94e74-928a-11ed-b70a-df639b6bea53'))

In [16]:
sql = """
        CREATE TABLE if NOT EXISTS news_opinion_sbert_embedding (
            time timestamp,
            news_uid uuid,
            sbert_embedding bytea,
            primary key (news_uid)
        );
        """
    
exec_sql(sql)

In [17]:
for news in tqdm.tqdm(all_opinion_news):
    time, title, content, uid = news
    content = create_validate_content(content)
    sbert_input = create_sbert_input(title, content)
    sbert_embedding = create_sbert_embedding(sbert_input)
    
    tensor_bytes = pickle.dumps(sbert_embedding)

    sql = f"""
        INSERT INTO news_opinion_sbert_embedding (time, news_uid, sbert_embedding) VALUES (%s, %s, %s);
        """
    
    exec_sql(sql, (time, uid, tensor_bytes))
    

  0%|          | 0/192 [00:00<?, ?it/s]

In [28]:
embeddings = [create_sbert_embedding(
                create_sbert_input(title,
                 create_validate_content(content))) for (time, title, content, uid) in tqdm.tqdm(all_opinion_news[:1])]

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:

# sql = """
#     SELECT sbert_embedding FROM news_opinion_sbert_embedding
#     WHERE news_uid = '00f94e74-928a-11ed-b70a-df639b6bea53';
# """

# result = exec_sql(sql)

# 3 step load vectors

In [36]:
def get_newsuid_vector_tuple_a_date():#from_date_include, to_date_exclude):
# date like '2023-01-01'
    # sql = f"""
    #     SELECT time, title, content, uid FROM news_data_select
    #     WHERE '{from_date_include}' <= time AND time < '{to_date_exclude}';
    #     """
    
    sql = f"""
        SELECT news_uid, sbert_embedding FROM news_opinion_sbert_embedding
        """
    result = exec_sql(sql)

    news_uid_list = [i[0] for i in result]
    sbert_embedding_list = [pickle.loads(i[1]) for i in result]
    return news_uid_list, sbert_embedding_list

In [37]:
news_uid_list, sbert_embedding_list = get_newsuid_vector_tuple_a_date()

In [38]:
news_uid_list[:3]

[UUID('00f94e74-928a-11ed-b70a-df639b6bea53'),
 UUID('010e08c3-8d3c-11ed-b9d3-a76b94595270'),
 UUID('01afead0-8c1f-11ed-b6b8-dda564eb3c91')]

In [39]:
sbert_embedding_list[:3]

[tensor([ 6.6586e-03,  8.6493e-02, -1.5517e-02,  7.9103e-02,  7.2034e-02,
         -8.3554e-02,  1.6418e-01, -1.0372e-02,  9.4317e-02, -2.5791e-02,
         -4.6232e-02, -3.0616e-02,  6.0030e-02, -1.5189e-01,  2.4357e-02,
         -9.7182e-02, -4.3418e-02,  1.1342e-01,  8.9891e-02,  1.4417e-02,
          9.9447e-02, -5.2651e-02,  1.4340e-01,  9.8463e-02,  5.8880e-02,
         -4.5817e-02, -6.5330e-03,  9.4902e-03,  5.0875e-02,  3.5241e-02,
          2.3463e-02,  3.4937e-02,  8.3110e-02,  8.5999e-02, -3.9629e-04,
         -6.7519e-02,  3.5899e-03, -1.9304e-02,  8.1324e-02,  1.5203e-01,
          1.1316e-01, -2.5537e-04, -1.3930e-01,  6.0254e-02, -3.5815e-02,
          1.1273e-01, -6.4767e-02,  1.2531e-01,  7.9875e-02,  1.4020e-01,
          1.1719e-02,  4.3657e-02,  2.5755e-02, -2.9574e-02, -2.3494e-02,
         -2.4767e-01, -3.0312e-02,  1.8624e-02,  5.5814e-02, -4.4308e-02,
          1.9954e-02, -6.4879e-02,  4.8547e-02, -2.3924e-02, -1.8495e-01,
         -8.4486e-02,  1.7606e-02,  8.

In [40]:
sbert_embedding_list[0].shape

torch.Size([768])

In [44]:
from sentence_transformers import util
import torch
import numpy as np
from sklearn.cluster import AgglomerativeClustering


sbert_embeddings = torch.stack(sbert_embedding_list)
sbert_embeddings = sbert_embeddings / np.linalg.norm(sbert_embeddings, axis=1, keepdims=True)

clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(sbert_embeddings)
cluster_assignment = clustering_model.labels_


In [46]:
len(cluster_assignment)

192

In [61]:
str(cluster_assignment[0])

'14'

In [74]:
from datetime import datetime
def cluster_result_to_db(news_uid_list, cluster_assignment, cluster_time:datetime):
    assert len(news_uid_list) == len(cluster_assignment)

    sql = """
        CREATE TABLE if NOT EXISTS news_cluster_result_testing (
            cluster_time timestamp,
            cluster_id text,
            news_uid_in_cluster text[]
        );
        SET TIME ZONE 'Asia/Taipei';
        """
    
    exec_sql(sql)
    

    cluster_result = {}
    for news_uid, cluster_id in zip(news_uid_list, cluster_assignment):
        cluster_id = str(cluster_id)
        if cluster_id not in cluster_result:
            
            cluster_result[cluster_id] = []
        cluster_result[cluster_id].append(news_uid)

    for cluster_id in cluster_result.keys():

        sql = f"""
            INSERT INTO news_cluster_result_testing (cluster_time, cluster_id, news_uid_in_cluster) VALUES (%s, %s, %s);
            """

        exec_sql(sql, (cluster_time, cluster_id, cluster_result[cluster_id]))

In [53]:
from datetime import date
cluster_time = date(2022, 3, 20)
cluster_time

datetime.date(2022, 3, 20)

In [75]:
cluster_result_to_db(news_uid_list, cluster_assignment, cluster_time)